In [1]:
from openai import OpenAI

In [2]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
import os

In [6]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [7]:
from datetime import datetime

In [8]:
import requests

In [9]:
from supabase import create_client, Client

In [10]:
supabase_url = os.environ.get("SUPABASE_URL")

In [11]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [12]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [13]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
.gte('news_date', '2026-02-08')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 282,
  'news_output': 'Crypto.com CEO Kris Marszalek is launching ai.com, a consumer-focused autonomous/agentic AI platform that allows users to create agents to perform tasks on their behalf. Reports say the ai.com domain was purchased for about $70 million and the platform will debut with a Super Bowl LX advertisement; the launch is reported to go live Feb 8, 2026, with free and paid subscription tiers and plans for features like financial integrations, agent marketplaces, and social components.',
  'source_urls': ['https://www.pcmag.com/news/aicom-domain-name-sells-for-70-million-set-to-launch-at-super-bowl-lx'],
  'news_date': '2026-02-08'},
 {'id': 283,
  'news_output': 'Crypto.com CEO Kris Marszalek is launching ai.com, a consumer-focused autonomous/agentic AI platform that allows users to create agents to perform tasks on their behalf. Reports say the ai.com domain was purchased for about $70 million and the platform will debut with a Super Bowl LX advertisement; the lau

In [21]:
news_data = [{'id': 299,
  'news_output': 'Additional reports expand the OpenAI earbuds rumor with new naming details: outlets cite a code-name "Sweetpea" and say the earbuds may be sold under a consumer name "Dime." Coverage emphasizes these are leaks/rumors (no official OpenAI confirmation) and mentions possible product scaling/backing concerns (e.g., memory costs).',
  'source_urls': ['https://wccftech.com/openais-sweetpea-earbuds-to-be-sold-under-the-consumer-name-dime-now-expected-to-chronically-underdeliver-on-the-hype/'],
  'news_date': '2026-02-08'}]


news_data[0]

{'id': 299,
 'news_output': 'Additional reports expand the OpenAI earbuds rumor with new naming details: outlets cite a code-name "Sweetpea" and say the earbuds may be sold under a consumer name "Dime." Coverage emphasizes these are leaks/rumors (no official OpenAI confirmation) and mentions possible product scaling/backing concerns (e.g., memory costs).',
 'source_urls': ['https://wccftech.com/openais-sweetpea-earbuds-to-be-sold-under-the-consumer-name-dime-now-expected-to-chronically-underdeliver-on-the-hype/'],
 'news_date': '2026-02-08'}

In [22]:
def prompt_generator_research():
    formatted_prompt = []
    
    for item in news_data:
        event_id = f"{item['id']}_{item['news_date']}"
        prompt = f"""You are given news summary: {item['news_output']} and source: {item['source_urls'][0]}, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources."""

        prompt_dict = {
            'event_id': event_id,
            'prompt': prompt
        }
        
        formatted_prompt.append(prompt_dict)

    return formatted_prompt

In [23]:
prompt_generator_research()

[{'event_id': '299_2026-02-08',
  'prompt': 'You are given news summary: Additional reports expand the OpenAI earbuds rumor with new naming details: outlets cite a code-name "Sweetpea" and say the earbuds may be sold under a consumer name "Dime." Coverage emphasizes these are leaks/rumors (no official OpenAI confirmation) and mentions possible product scaling/backing concerns (e.g., memory costs). and source: https://wccftech.com/openais-sweetpea-earbuds-to-be-sold-under-the-consumer-name-dime-now-expected-to-chronically-underdeliver-on-the-hype/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.'}]

In [24]:
def openai_research():
    research_output = []
    for i in system_prompt:
        
        response = client.responses.create(
            model = "gpt-4o",
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = i['prompt'],
        )

        output = response.output_text

        final_dict = {
            'event_id': i['event_id'],
            'output': output
        }

        research_output.append(final_dict)

    return research_output

In [27]:
def openai_research_v2():
    research_output = []
    for i in news_data:
        response = client.responses.create(
            model = "gpt-4o",
            tools = [{
                "type": "web_search"
            }],
            include = ["web_search_call.action.sources"],
            input = f"""You are an AI research analyst for Krux.

            Your job is to expand a given AI news summary into a structured, fact-checked research brief.

            CRITICAL REQUIREMENTS:
            - You must ONLY include factual information.
            - DO NOT include opinions, interpretations, predictions, or analysis.
            - Every fact MUST be traceable to a credible published source.
            - If a fact cannot be verified from a reliable source, DO NOT include it.
            - Never assume or infer missing information.

            TASK:
            1. Validate the claims made in the summary.
            2. Expand with additional confirmed facts.
            3. Provide supporting citations for every statement.


            Here are the details of the news that you need to research on:

            {i['news_output']}
            """,
        )

        output = response.output_text

        research_output.append(output)

    print(research_output)

In [ ]:
openai_research_v2()